In [1]:
import pandas as pd

df = pd.read_csv('data/NAM-Scrips.csv')

exclude = ['KPITTECH','AIRTELPP','TATASTLLP','MTARTECH','NAVKARCORP','SEYAIND']

df = df.dropna()

stock_ids = df[~df['NSE ID'].isin(exclude) ]['NSE ID'].unique()


In [2]:
stock_ids

array(['DEEPAKNTR', 'TIINDIA', 'BIRLACORPN', 'NAVINFLUOR', 'ORIENTELEC',
       'NIITLTD', 'RADICO', 'BAJAJELEC', 'DIXON', 'BALRAMCHIN',
       'CARBORUNIV', 'CDSL', 'STLTECH', 'TEJASNET', 'PERSISTENT',
       'ECLERX', 'POONAWALLA', 'CYIENT', 'AFFLE', 'HONAUT', 'KPRMILL',
       'TIMKEN', 'FINEORG', 'LUXIND', 'LT', 'POLYPLEX', 'IEX',
       'ZYDUSWELL', 'KECL', 'HGINFRA', 'BEL', 'CHAMBLFERT', 'CREDITACC',
       'BANKBARODA', 'RALLIS', 'TATACONSUM', 'KIRLOSBROS', 'BSOFT', 'SIS',
       'SBIN', 'FINCABLES', 'BAYERCROP', 'JYOTHYLAB', 'AXISBANK', 'FDC',
       'MCDOWELL-N', 'ZENSARTECH', 'SUNPHARMA', 'MAHINDCIE', 'CENTURYTEX',
       'HONDAPOWER', 'ADANIPORTS', 'VSTTILLERS', 'INDOCO', 'ISGEC', 'MCX',
       'VOLTAMP', 'WSTCSTPAPR', 'GALAXYSURF', 'RAYMOND', 'STOVEKRAFT',
       'VINDHYATEL', 'MAHLOG', 'KALPATPOWR', 'VIPIND', 'LINDEINDIA',
       'RBLBANK', 'GOKEX', 'APARINDS', 'GENUSPOWER', 'HDFCLIFE',
       'AUTOAXLES', 'VTL', 'BAJAJCON', 'TDPOWERSYS', 'GRINDWELL', 'PGHL',
       'HAL',

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

df_analysis = pd.DataFrame()

for stock_name in stock_ids: 

#stock_name = 'tcs'

    print(stock_name)

    link_blueprint = "https://www.screener.in/company/"
    
    actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"
    
    def get_list(actual_link):
        page = requests.get(actual_link)
        link_soup = BeautifulSoup(page.content, 'html.parser')
        
        test_list = link_soup.find(id="quarters").text.replace("  ", "").split('\n')

        while("" in test_list) :
            test_list.remove("")
        while("\r" in test_list) :
            test_list.remove("\r")        
        while(" \r" in test_list) :
            test_list.remove(" \r")
            
        return test_list
    
    test_list = get_list(actual_link)
        
    print(len(test_list))
    n = len(test_list) 
    
    if n < 114:
        actual_link = link_blueprint + stock_name.upper() 
        test_list = get_list(actual_link)
        print(len(test_list))
    
    q = 0
        
    if n > 162:
        p = 8
        q = n-162
    elif n > 120 and n <= 162:
        p = 8
    elif n < 120 and n > 100:
        p = 7   
        
    else:
        p = 6

    df_qreults = pd.DataFrame()
    
    if 'Gross NPA %'  in test_list:
        endkey = 'Gross NPA %'
    else:
        endkey = 'Raw PDF'

    if 'Dec 2021' in test_list:
        quarter = 'Dec 2021'
    else:
        quarter = 'Sep 2021'

    #df_qreults['Quarter'] = test_list[test_list.index('Sep 2021')-p:test_list.index('Sep 2021')+1]
    #df_qreults['EPS(Q)'] = test_list[test_list.index('Raw PDF')-p-q-1:test_list.index('Raw PDF')-q]
    s = test_list.index(endkey)  - test_list.index('EPS in Rs')-2
    df_qreults['Quarter'] = test_list[test_list.index(quarter)-s:test_list.index(quarter)+1]    
    df_qreults['EPS(Q)'] = test_list[test_list.index('EPS in Rs')+1:test_list.index(endkey)]

    try:
        df_qreults['Sales(Q)'] = test_list[test_list.index('Expenses\xa0+')-s-1:test_list.index('Expenses\xa0+')]
    except:
        df_qreults['Sales(Q)'] = test_list[test_list.index('Interest')-s-1:test_list.index('Interest')] 
    
    
    df_qreults['Sales(Q)']  = df_qreults['Sales(Q)'].str.replace(",", "")

    df_qreults['EPS(Q)'] = df_qreults['EPS(Q)'].astype(float).fillna(0.0)
    df_qreults['Sales(Q)'] = df_qreults['Sales(Q)'].astype(float).fillna(0.0)

    from pandas.tseries.offsets import MonthEnd
    import datetime

    df_qreults['Quarter'] = pd.to_datetime(df_qreults['Quarter'], format="%b %Y") + MonthEnd(1) + datetime.timedelta(days=46)
    df_qreults

    df_qreults['EPS(TTM)'] = df_qreults['EPS(Q)'] + df_qreults['EPS(Q)'].shift(1) + df_qreults['EPS(Q)'].shift(2) + \
                                df_qreults['EPS(Q)'].shift(3)
    
    df_qreults['Sales(TTM)'] = df_qreults['Sales(Q)'] + df_qreults['Sales(Q)'].shift(1) + df_qreults['Sales(Q)'].shift(2) + \
                                df_qreults['Sales(Q)'].shift(3)

    import pandas_datareader.data as web
    import yfinance as yfin
    yfin.pdr_override()

    df_stock = pd.DataFrame()

    for date in df_qreults['Quarter']:
        print(date)

        end_date = date
        start_date = (date + datetime.timedelta(days=-5)).strftime('%Y-%m-%d')


        stock_id = stock_name.upper() + '.NS'
        tmp = pd.DataFrame(web.DataReader(tickers = stock_id, data_source='yahoo', start = start_date, end = end_date))
        tmp = tmp.tail(1)
        tmp['Stock'] = stock_id
        tmp['Quarter'] = end_date  
        tmp['Close'] = round(tmp['Close'],2)
        tmp.reset_index(inplace = True)
        cols = ['Stock','Quarter','Close']
        tmp =tmp[cols]    
        df_stock = df_stock.append(tmp)

    df_stock_analysis = df_stock.merge(df_qreults, on='Quarter').tail(6)
    df_stock_analysis['PE(TTM)'] = round(df_stock_analysis['Close']/df_stock_analysis['EPS(TTM)'],2)
    
    df_analysis = df_analysis.append(df_stock_analysis)
    
df_analysis

DEEPAKNTR
162
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11

[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
DIXON
160
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[**********

[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
POONAWALLA
160
2019-02-15 00:00:00
[*****

[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
LUXIND
160
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********

[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
CHAMBLFERT
160
2019-02-15 00:00:00
[*****

...

[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
KEC
160
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[************

[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
VESUVIUS
14
158
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[****

[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
INDHOTEL
160
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*******

[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed


,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM)
6,DEEPAKNTR.NS,2020-08-15,585.70,7.25,674.0,42.40,3853.0,13.81
7,DEEPAKNTR.NS,2020-11-15,822.50,12.48,987.0,43.85,3837.0,18.76
8,DEEPAKNTR.NS,2021-02-15,1110.65,15.88,1235.0,48.24,3952.0,23.02
9,DEEPAKNTR.NS,2021-05-16,1728.10,21.27,1463.0,56.88,4359.0,30.38
10,DEEPAKNTR.NS,2021-08-15,2142.45,22.19,1526.0,71.82,5211.0,29.83
...,...,...,...,...,...,...,...,...
4,GREENPANEL.NS,2020-11-15,83.90,1.50,225.0,-0.82,789.0,-102.32
5,GREENPANEL.NS,2021-02-15,172.80,2.48,317.0,1.62,870.0,106.67
6,GREENPANEL.NS,2021-05-16,240.20,4.60,389.0,5.60,1021.0,42.89
7,GREENPANEL.NS,2021-08-15,247.10,2.43,308.0,11.01,1239.0,22.44


In [4]:
df_analysis_final = pd.DataFrame()
for stock_id in df_analysis['Stock'].unique():
    tmp = df_analysis[df_analysis['Stock'] == stock_id]
    tmp['Prev_EPS(Q)'] = tmp['EPS(Q)'].shift(1)
    tmp['Prev_EPS(TTM)'] = tmp['EPS(TTM)'].shift(1)
    tmp['Prev_Sales'] = tmp['Sales(Q)'].shift(1)
    tmp['Prev_PE'] = tmp['PE(TTM)'].shift(1)
    tmp['Prev_Close'] = tmp['Close'].shift(1)
    tmp['FR%'] = round((tmp['EPS(TTM)'] - tmp['Prev_EPS(TTM)'])*100/tmp['Prev_EPS(TTM)'],2)
    tmp['SR%'] = round(tmp['EPS(TTM)']*(tmp['PE(TTM)'] - tmp['Prev_PE'])*100/tmp['Prev_Close'],2)
    tmp['TR%'] = tmp['FR%'] + tmp['SR%']
    tmp['Sales%'] = round((tmp['Sales(Q)'] - tmp['Prev_Sales'])*100/tmp['Prev_Sales'],2)
    tmp['EPS%'] = round((tmp['EPS(Q)'] - tmp['Prev_EPS(Q)'])*100/tmp['Prev_EPS(Q)'],2)
    tmp['Return%'] = tmp['FR%'] + tmp['SR%']
    df_analysis_final = df_analysis_final.append(tmp)
    
df_analysis_final

<ipython-input-4-e22d75f1c612>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['Prev_EPS(Q)'] = tmp['EPS(Q)'].shift(1)
<ipython-input-4-e22d75f1c612>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['Prev_EPS(TTM)'] = tmp['EPS(TTM)'].shift(1)
<ipython-input-4-e22d75f1c612>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM),Prev_EPS(Q),Prev_EPS(TTM),Prev_Sales,Prev_PE,Prev_Close,FR%,SR%,TR%,Sales%,EPS%,Return%
6,DEEPAKNTR.NS,2020-08-15,585.70,7.25,674.0,42.40,3853.0,13.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,DEEPAKNTR.NS,2020-11-15,822.50,12.48,987.0,43.85,3837.0,18.76,7.25,42.40,674.0,13.81,585.70,3.42,37.06,40.48,46.44,72.14,40.48
8,DEEPAKNTR.NS,2021-02-15,1110.65,15.88,1235.0,48.24,3952.0,23.02,12.48,43.85,987.0,18.76,822.50,10.01,24.99,35.00,25.13,27.24,35.00
9,DEEPAKNTR.NS,2021-05-16,1728.10,21.27,1463.0,56.88,4359.0,30.38,15.88,48.24,1235.0,23.02,1110.65,17.91,37.69,55.60,18.46,33.94,55.60
10,DEEPAKNTR.NS,2021-08-15,2142.45,22.19,1526.0,71.82,5211.0,29.83,21.27,56.88,1463.0,30.38,1728.10,26.27,-2.29,23.98,4.31,4.33,23.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,GREENPANEL.NS,2020-11-15,83.90,1.50,225.0,-0.82,789.0,-102.32,-2.98,-2.01,90.0,-22.09,44.40,-59.20,148.17,88.97,150.00,-150.34,88.97
5,GREENPANEL.NS,2021-02-15,172.80,2.48,317.0,1.62,870.0,106.67,1.50,-0.82,225.0,-102.32,83.90,-297.56,403.53,105.97,40.89,65.33,105.97
6,GREENPANEL.NS,2021-05-16,240.20,4.60,389.0,5.60,1021.0,42.89,2.48,1.62,317.0,106.67,172.80,245.68,-206.69,38.99,22.71,85.48,38.99
7,GREENPANEL.NS,2021-08-15,247.10,2.43,308.0,11.01,1239.0,22.44,4.60,5.60,389.0,42.89,240.20,96.61,-93.74,2.87,-20.82,-47.17,2.87


In [5]:
cols = ['Stock','Quarter','Sales%','EPS%','Return%']
df_growth = df_analysis_final[cols]
df_growth[df_growth['Quarter'] == '2021-11-15'].sort_values(by='Sales%', ascending=False).head(20)

,Stock,Quarter,Sales%,EPS%,Return%
11,HAL.NS,2021-11-15,243.50,335.05,30.12
11,BEL.NS,2021-11-15,123.04,2460.00,28.15
11,INDHOTEL.NS,2021-11-15,111.01,-56.67,50.65
11,BRIGADE.NS,2021-11-15,96.61,-129.71,52.70
11,RAYMOND.NS,2021-11-15,87.77,-134.03,20.93
11,GOKEX.NS,2021-11-15,84.23,-1216.95,14.39
9,AFFLE.NS,2021-11-15,80.92,33.21,39.24
2,STOVEKRAFT.NS,2021-11-15,69.63,69.64,NaN
11,WSTCSTPAPR.NS,2021-11-15,67.29,126.65,0.49
11,VOLTAMP.NS,2021-11-15,62.35,122.10,16.16


In [6]:
df_analysis_final.to_csv('data/NAM_FvS_QoQ_Analysis.csv', index=False)

In [7]:
df_growth = pd.read_csv('data/NAM_FvS_QoQ_Analysis.csv')
cols = ['Stock','Quarter','Sales%','EPS%','Return%']
#df_growth = df_growth[cols]
df_growth = df_growth.dropna()
df_growth

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM),Prev_EPS(Q),Prev_EPS(TTM),Prev_Sales,Prev_PE,Prev_Close,FR%,SR%,TR%,Sales%,EPS%,Return%
1,DEEPAKNTR.NS,2020-11-15,822.50,12.48,987.0,43.85,3837.0,18.76,7.25,42.40,674.0,13.81,585.70,3.42,37.06,40.48,46.44,72.14,40.48
2,DEEPAKNTR.NS,2021-02-15,1110.65,15.88,1235.0,48.24,3952.0,23.02,12.48,43.85,987.0,18.76,822.50,10.01,24.99,35.00,25.13,27.24,35.00
3,DEEPAKNTR.NS,2021-05-16,1728.10,21.27,1463.0,56.88,4359.0,30.38,15.88,48.24,1235.0,23.02,1110.65,17.91,37.69,55.60,18.46,33.94,55.60
4,DEEPAKNTR.NS,2021-08-15,2142.45,22.19,1526.0,71.82,5211.0,29.83,21.27,56.88,1463.0,30.38,1728.10,26.27,-2.29,23.98,4.31,4.33,23.98
5,DEEPAKNTR.NS,2021-11-15,2354.25,18.65,1681.0,77.99,5905.0,30.19,22.19,71.82,1526.0,29.83,2142.45,8.59,1.31,9.90,10.16,-15.95,9.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,GREENPANEL.NS,2020-11-15,83.90,1.50,225.0,-0.82,789.0,-102.32,-2.98,-2.01,90.0,-22.09,44.40,-59.20,148.17,88.97,150.00,-150.34,88.97
712,GREENPANEL.NS,2021-02-15,172.80,2.48,317.0,1.62,870.0,106.67,1.50,-0.82,225.0,-102.32,83.90,-297.56,403.53,105.97,40.89,65.33,105.97
713,GREENPANEL.NS,2021-05-16,240.20,4.60,389.0,5.60,1021.0,42.89,2.48,1.62,317.0,106.67,172.80,245.68,-206.69,38.99,22.71,85.48,38.99
714,GREENPANEL.NS,2021-08-15,247.10,2.43,308.0,11.01,1239.0,22.44,4.60,5.60,389.0,42.89,240.20,96.61,-93.74,2.87,-20.82,-47.17,2.87


In [8]:
df_growth[(df_growth['Sales%'] > 5) & (df_growth['EPS%'] > 5)]['Stock'].value_counts()[0:20]

FINEORG.NS       4
KEI.NS           4
HAL.NS           4
ECLERX.NS        4
FINCABLES.NS     4
LUXIND.NS        4
LT.NS            3
TIINDIA.NS       3
VESUVIUS.NS      3
APEX.NS          3
PERSISTENT.NS    3
DEEPAKNTR.NS     3
ORIENTCEM.NS     3
AFFLE.NS         3
KEC.NS           3
DIXON.NS         3
PTC.NS           3
VTL.NS           3
JKLAKSHMI.NS     3
VOLTAMP.NS       3
Name: Stock, dtype: int64

In [9]:
df_tmp_final = pd.DataFrame()
for stock in df_growth['Stock'].unique():
    df_tmp = df_growth[df_growth['Stock'] == stock][cols]
    df_tmp['sales_flag'] = 0
    df_tmp['eps_flag'] = 0
    df_tmp.loc[df_tmp['Sales%'] < 0, 'sales_flag'] = 1
    df_tmp.loc[df_tmp['EPS%'] < 0, 'eps_flag'] = 1
    df_tmp['sales_weight'] =  df_tmp['sales_flag']*5 + df_tmp['sales_flag'].shift(1)*4 + df_tmp['sales_flag'].shift(2)*3+df_tmp['sales_flag'].shift(3)*2+df_tmp['sales_flag'].shift(4)*1
    df_tmp['eps_weight'] =  df_tmp['eps_flag']*5 + df_tmp['eps_flag'].shift(1)*4 + df_tmp['eps_flag'].shift(2)*3+df_tmp['eps_flag'].shift(3)*2+df_tmp['eps_flag'].shift(4)*1
    df_tmp['total_weight'] = df_tmp['sales_weight'] + df_tmp['eps_weight'] 
    df_tmp = df_tmp.tail(1)
    df_tmp_final = df_tmp_final.append(df_tmp)

In [10]:
df_tmp_final[(df_tmp_final['Sales%'] > 5) & (df_tmp_final['EPS%'] > 5)].sort_values(by = 'total_weight', ascending=True)

,Stock,Quarter,Sales%,EPS%,Return%,sales_flag,eps_flag,sales_weight,eps_weight,total_weight
89,PERSISTENT.NS,2021-11-15,9.84,6.92,32.73,0,0,0.0,0.0,0.0
137,FINEORG.NS,2021-11-15,22.35,38.94,13.90,0,0,0.0,2.0,2.0
473,INTELLECT.NS,2021-11-15,10.78,7.26,8.60,0,0,0.0,4.0,4.0
161,IEX.NS,2021-11-15,20.88,22.86,93.71,0,0,4.0,0.0,4.0
107,CYIENT.NS,2021-11-15,5.10,5.46,15.23,0,0,4.0,0.0,4.0
95,ECLERX.NS,2021-11-15,7.61,10.32,9.09,0,0,0.0,4.0,4.0
431,VTL.NS,2021-11-15,23.77,52.90,12.17,0,0,4.0,1.0,5.0
71,CDSL.NS,2021-11-15,24.79,34.53,19.11,0,0,2.0,3.0,5.0
624,PTC.NS,2021-11-15,10.39,47.29,23.60,0,0,2.0,5.0,7.0
612,AJANTPHARM.NS,2021-11-15,18.32,12.75,-2.86,0,0,4.0,3.0,7.0


In [12]:
df_growth[df_growth['Stock'] == 'INTELLECT.NS'][cols]

,Stock,Quarter,Sales%,EPS%,Return%
469,INTELLECT.NS,2020-11-15,7.51,38.82,36.77
470,INTELLECT.NS,2021-02-15,2.69,35.57,73.69
471,INTELLECT.NS,2021-05-16,4.19,0.00,61.35
472,INTELLECT.NS,2021-08-15,2.51,-9.08,-7.17
473,INTELLECT.NS,2021-11-15,10.78,7.26,8.60
